In [1]:
import os
import re
import random
import copy
from collections import Counter
try:
    import cPickle as pickle
except ImportError:
    import pickle
import gc
    
from gensim.models import KeyedVectors

import numpy as np
from numpy.random import seed
seed(1000)
from tensorflow.random import set_random_seed
set_random_seed(1000)

import pandas
from pandas import Series, DataFrame

import keras
import keras.backend as K
# from keras.layers import Layer
from keras.engine import Layer
from keras.engine import InputSpec
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.layers import Input, Embedding, Conv1D, Concatenate, Flatten, Dropout, Dense, Bidirectional, Activation, GRU
from keras.layers import MaxPooling1D, Conv2D, Flatten, Reshape, MaxPooling2D, LSTM, Lambda, Highway, Dot, Permute, Add
from keras.models import Model, Sequential
from keras.activations import softmax
from keras.initializers import Constant, RandomNormal, RandomUniform

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

import matplotlib.pyplot as plt
import seaborn as sns       

from scipy import stats
from scipy.stats import  norm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 

#进行配置，使用30%的GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
session = tf.Session(config=config)

# 设置session
KTF.set_session(session )

Using TensorFlow backend.


In [2]:
# 文件路径
ORIGINAL_FILE_PATH = "../data/original_data/quora_duplicate_questions.tsv"

DATA_ALL_PATH = "../data/all_data.txt"

DATA_TRAIN_PATH = "../data/train.txt"
DATA_VALID_PATH = "../data/valid.txt"
DATA_TEST_PATH = "../data/test.txt"

MAPS_FILE_PATH = "../data/maps.pkl"

# Glove文件
WORD2VEC_PATH = "../data/GoogleNews-vectors-negative300.bin"

# Model Path
MODEL_PATH = "../model/mvlstm-capsnet/mvlstm-capsnet-01.h5"

# 创建文件夹
(file_path, file_name) = os.path.split(MODEL_PATH)
if not os.path.exists(file_path):
    os.mkdir(file_path)

In [3]:
def load_dict(dict_path):
    with open(dict_path, 'rb') as fr:
        return pickle.load(fr)
        
def create_maps(dico):
    item2id = dico
    id2item = dict([val, key] for key, val in dico.items())
    return item2id, id2item

def read_file(fpath):
    """
    读取文件，返回
    """
    question1, question2, labels = [], [], []
    with open(fpath, 'r', encoding="utf-8") as fr:
        for line in fr.readlines():
            try:
                q, k, l = line.strip().split('\t')
            except ValueError:
                print(line)
            question1.append(q)
            question2.append(k)
            labels.append(int(l))
            
    return question1, question2, labels

def load_data(fpath, word_to_id, max_length=20):
    """
    载入数据，并将其转化为id表示
    """
    question1, question2, labels = read_file(fpath)
    
    q1_id, q2_id = [], []
    for i in range(len(question1)):
        q1= question1[i].split()
        q2 = question2[i].split()
        q1_id.append([word_to_id[x] for x in q1 if x in word_to_id])
        q2_id.append([word_to_id[x] for x in q2 if x in word_to_id])

    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x1_pad = keras.preprocessing.sequence.pad_sequences(q1_id, max_length, padding="post")
    x2_pad = keras.preprocessing.sequence.pad_sequences(q2_id, max_length, padding="post")
    labels = keras.utils.np_utils.to_categorical(labels)
    
    return x1_pad, x2_pad, labels
    
        
def load_word2vec(embedding_path, word_index, embed_dim=300): 
    """
    载入预训练好的word2vec(Google-News)
    """
    # 载入word2vec词向量
    word2vec_dict = KeyedVectors.load_word2vec_format(embedding_path,binary=True)
    
    embedding_index = dict()
    for word in word2vec_dict.wv.vocab:
        embedding_index[word] = word2vec_dict.word_vec(word)
    print('Load %s word vectors.' % len(embedding_index))
    
    all_embs = np.stack(list(embedding_index.values()))
    # emb_mean, emb_std = all_embs.mean(), all_embs.std()
    emb_mean = np.mean(all_embs, axis=0)
    vocab_size = len(word_index)
    # 初始化权重
    embedding_matrix = np.zeros((vocab_size+1, embed_dim))
    gc.collect()
    # 对权重矩阵进行赋值，未找到时用词向量平均值填充
    for word, i in word_index.items():
        if i < vocab_size + 1:
            embedding_vector = embedding_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
            else:
                embedding_matrix[i] = emb_mean
    del embedding_index
    return embedding_matrix

In [4]:
dico = load_dict(MAPS_FILE_PATH)
word_index, index_word = create_maps(dico)

# 准备数据
x1_train, x2_train, y_train = load_data(DATA_TRAIN_PATH, word_index)
x1_valid, x2_valid, y_valid = load_data(DATA_VALID_PATH, word_index)
x1_test, x2_test, y_test = load_data(DATA_TEST_PATH, word_index)

embedding_matrix = load_word2vec(WORD2VEC_PATH, word_index)

Load 3000000 word vectors.


In [5]:
"""
Some key layers used for constructing a Capsule Network. These layers can used to construct CapsNet on other dataset, 
not just on MNIST.
*NOTE*: some functions can be implemented in multiple ways, I keep all of them. You can try them for yourself just by
uncommenting them and commenting their counterparts.

Author: Xifeng Guo, E-mail: `guoxifeng1990@163.com`, Github: `https://github.com/XifengGuo/CapsNet-Keras`
"""

import keras.backend as K
import tensorflow as tf
from keras import initializers, layers


class Length(layers.Layer):
    """
    Compute the length of vectors. This is used to compute a Tensor that has the same shape with y_true in margin_loss.
    Using this layer as model's output can directly predict labels by using `y_pred = np.argmax(model.predict(x), 1)`
    inputs: shape=[None, num_vectors, dim_vector]
    output: shape=[None, num_vectors]
    """
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1))

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]


class Mask(layers.Layer):
    """
    Mask a Tensor with shape=[None, num_capsule, dim_vector] either by the capsule with max length or by an additional 
    input mask. Except the max-length capsule (or specified capsule), all vectors are masked to zeros. Then flatten the
    masked Tensor.
    For example:
        ```
        x = keras.layers.Input(shape=[8, 3, 2])  # batch_size=8, each sample contains 3 capsules with dim_vector=2
        y = keras.layers.Input(shape=[8, 3])  # True labels. 8 samples, 3 classes, one-hot coding.
        out = Mask()(x)  # out.shape=[8, 6]
        # or
        out2 = Mask()([x, y])  # out2.shape=[8,6]. Masked with true labels y. Of course y can also be manipulated.
        ```
    """
    def call(self, inputs, **kwargs):
        if type(inputs) is list:  # true label is provided with shape = [None, n_classes], i.e. one-hot code.
            assert len(inputs) == 2
            inputs, mask = inputs
        else:  # if no true label, mask by the max length of capsules. Mainly used for prediction
            # compute lengths of capsules
            x = K.sqrt(K.sum(K.square(inputs), -1))
            # generate the mask which is a one-hot code.
            # mask.shape=[None, n_classes]=[None, num_capsule]
            mask = K.one_hot(indices=K.argmax(x, 1), num_classes=x.get_shape().as_list()[1])

        # inputs.shape=[None, num_capsule, dim_capsule]
        # mask.shape=[None, num_capsule]
        # masked.shape=[None, num_capsule * dim_capsule]
        masked = K.batch_flatten(inputs * K.expand_dims(mask, -1))
        return masked

    def compute_output_shape(self, input_shape):
        if type(input_shape[0]) is tuple:  # true label provided
            return tuple([None, input_shape[0][1] * input_shape[0][2]])
        else:  # no true label provided
            return tuple([None, input_shape[1] * input_shape[2]])


def squash(vectors, axis=-1):
    """
    The non-linear activation used in Capsule. It drives the length of a large vector to near 1 and small vector to 0
    :param vectors: some vectors to be squashed, N-dim tensor
    :param axis: the axis to squash
    :return: a Tensor with same shape as input vectors
    """
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors


class CapsuleLayer(layers.Layer):
    """
    The capsule layer. It is similar to Dense layer. Dense layer has `in_num` inputs, each is a scalar, the output of the 
    neuron from the former layer, and it has `out_num` output neurons. CapsuleLayer just expand the output of the neuron
    from scalar to vector. So its input shape = [None, input_num_capsule, input_dim_capsule] and output shape = \
    [None, num_capsule, dim_capsule]. For Dense Layer, input_dim_capsule = dim_capsule = 1.
    
    :param num_capsule: number of capsules in this layer
    :param dim_capsule: dimension of the output vectors of the capsules in this layer
    :param routings: number of iterations for the routing algorithm
    """
    def __init__(self, num_capsule, dim_capsule, routings=3,
                 kernel_initializer='glorot_uniform',
                 **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_initializer = initializers.get(kernel_initializer)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "The input Tensor should have shape=[None, input_num_capsule, input_dim_capsule]"
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]

        # Transform matrix
        self.W = self.add_weight(shape=[self.num_capsule, self.input_num_capsule,
                                        self.dim_capsule, self.input_dim_capsule],
                                 initializer=self.kernel_initializer,
                                 name='W')

        self.built = True

    def call(self, inputs, training=None):
        # inputs.shape=[None, input_num_capsule, input_dim_capsule]
        # inputs_expand.shape=[None, 1, input_num_capsule, input_dim_capsule]
        inputs_expand = K.expand_dims(inputs, 1)

        # Replicate num_capsule dimension to prepare being multiplied by W
        # inputs_tiled.shape=[None, num_capsule, input_num_capsule, input_dim_capsule]
        inputs_tiled = K.tile(inputs_expand, [1, self.num_capsule, 1, 1])

        # Compute `inputs * W` by scanning inputs_tiled on dimension 0.
        # x.shape=[num_capsule, input_num_capsule, input_dim_capsule]
        # W.shape=[num_capsule, input_num_capsule, dim_capsule, input_dim_capsule]
        # Regard the first two dimensions as `batch` dimension,
        # then matmul: [input_dim_capsule] x [dim_capsule, input_dim_capsule]^T -> [dim_capsule].
        # inputs_hat.shape = [None, num_capsule, input_num_capsule, dim_capsule]
        inputs_hat = K.map_fn(lambda x: K.batch_dot(x, self.W, [2, 3]), elems=inputs_tiled)

        # Begin: Routing algorithm ---------------------------------------------------------------------#
        # The prior for coupling coefficient, initialized as zeros.
        # b.shape = [None, self.num_capsule, self.input_num_capsule].
        b = tf.zeros(shape=[K.shape(inputs_hat)[0], self.num_capsule, self.input_num_capsule])

        assert self.routings > 0, 'The routings should be > 0.'
        for i in range(self.routings):
            # c.shape=[batch_size, num_capsule, input_num_capsule]
            c = tf.nn.softmax(b, dim=1)

            # c.shape =  [batch_size, num_capsule, input_num_capsule]
            # inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule]
            # The first two dimensions as `batch` dimension,
            # then matmal: [input_num_capsule] x [input_num_capsule, dim_capsule] -> [dim_capsule].
            # outputs.shape=[None, num_capsule, dim_capsule]
            outputs = squash(K.batch_dot(c, inputs_hat, [2, 2]))  # [None, 10, 16]

            if i < self.routings - 1:
                # outputs.shape =  [None, num_capsule, dim_capsule]
                # inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule]
                # The first two dimensions as `batch` dimension,
                # then matmal: [dim_capsule] x [input_num_capsule, dim_capsule]^T -> [input_num_capsule].
                # b.shape=[batch_size, num_capsule, input_num_capsule]
                b += K.batch_dot(outputs, inputs_hat, [2, 3])
        # End: Routing algorithm -----------------------------------------------------------------------#

        return outputs

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_capsule])


def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
    """
    Apply Conv2D `n_channels` times and concatenate all capsules
    :param inputs: 4D tensor, shape=[None, width, height, channels]
    :param dim_capsule: the dim of the output vector of capsule
    :param n_channels: the number of types of capsules
    :return: output tensor, shape=[None, num_capsule, dim_capsule]
    """
    output = Conv1D(filters=dim_capsule*n_channels, kernel_size=kernel_size, strides=strides, padding=padding,
                           name='primarycap_conv1d')(inputs)
    outputs = Reshape(target_shape=[-1, dim_capsule], name='primarycap_reshape')(output)
    return layers.Lambda(squash, name='primarycap_squash')(outputs)


"""
# The following is another way to implement primary capsule layer. This is much slower.
# Apply Conv2D `n_channels` times and concatenate all capsules
def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
    outputs = []
    for _ in range(n_channels):
        output = layers.Conv2D(filters=dim_capsule, kernel_size=kernel_size, strides=strides, padding=padding)(inputs)
        outputs.append(layers.Reshape([output.get_shape().as_list()[1] ** 2, dim_capsule])(output))
    outputs = layers.Concatenate(axis=1)(outputs)
    return layers.Lambda(squash)(outputs)
"""


'\n# The following is another way to implement primary capsule layer. This is much slower.\n# Apply Conv2D `n_channels` times and concatenate all capsules\ndef PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):\n    outputs = []\n    for _ in range(n_channels):\n        output = layers.Conv2D(filters=dim_capsule, kernel_size=kernel_size, strides=strides, padding=padding)(inputs)\n        outputs.append(layers.Reshape([output.get_shape().as_list()[1] ** 2, dim_capsule])(output))\n    outputs = layers.Concatenate(axis=1)(outputs)\n    return layers.Lambda(squash)(outputs)\n'

In [6]:
import numpy as np

import keras
from keras import backend as K
from keras.engine import Layer
from keras.engine import InputSpec

class Match(Layer):
    """Layer that computes a matching matrix between samples in two tensors.
    # Arguments
        normalize: Whether to L2-normalize samples along the
            dot product axis before taking the dot product.
            If set to True, then the output of the dot product
            is the cosine proximity between the two samples.
        **kwargs: Standard layer keyword arguments.
    """

    def __init__(self, normalize=False, match_type='dot', **kwargs):
        super(Match, self).__init__(**kwargs)
        self.normalize = normalize
        self.match_type = match_type
        self.supports_masking = True
        if match_type not in ['dot', 'mul', 'plus', 'minus', 'concat']:
            raise ValueError('In `Match` layer, '
                             'param match_type=%s is unknown.' % match_type)

    def build(self, input_shape):
        # Used purely for shape validation.
        if not isinstance(input_shape, list) or len(input_shape) != 2:
            raise ValueError('A `Match` layer should be called '
                             'on a list of 2 inputs.')
        self.shape1 = input_shape[0]
        self.shape2 = input_shape[1]
        if self.shape1[0] != self.shape2[0]:
            raise ValueError(
                'Dimension incompatibility '
                '%s != %s. ' % (self.shape1[0], self.shape2[0]) +
                'Layer shapes: %s, %s' % (self.shape1, self.shape2))
        if self.shape1[2] != self.shape2[2]:
            raise ValueError(
                'Dimension incompatibility '
                '%s != %s. ' % (self.shape1[2], self.shape2[2]) +
                'Layer shapes: %s, %s' % (self.shape1, self.shape2))

    def call(self, inputs):
        x1 = inputs[0]
        x2 = inputs[1]
        if self.match_type in ['dot']:
            if self.normalize:
                x1 = K.l2_normalize(x1, axis=2)
                x2 = K.l2_normalize(x2, axis=2)
            output = K.tf.einsum('abd,acd->abc', x1, x2)
            output = K.tf.expand_dims(output, 3)
        elif self.match_type in ['mul', 'plus', 'minus']:
            x1_exp = K.tf.stack([x1] * self.shape2[1], 2)
            x2_exp = K.tf.stack([x2] * self.shape1[1], 1)
            if self.match_type == 'mul':
                output = x1_exp * x2_exp
            elif self.match_type == 'plus':
                output = x1_exp + x2_exp
            elif self.match_type == 'minus':
                output = x1_exp - x2_exp
        elif self.match_type in ['concat']:
            x1_exp = K.tf.stack([x1] * self.shape2[1], axis=2)
            x2_exp = K.tf.stack([x2] * self.shape1[1], axis=1)
            output = K.tf.concat([x1_exp, x2_exp], axis=3)

        return output

    def compute_output_shape(self, input_shape):
        if not isinstance(input_shape, list) or len(input_shape) != 2:
            raise ValueError('A `Match` layer should be called '
                             'on a list of 2 inputs.')
        shape1 = list(input_shape[0])
        shape2 = list(input_shape[1])
        if len(shape1) != 3 or len(shape2) != 3:
            raise ValueError('A `Match` layer should be called '
                             'on 2 inputs with 3 dimensions.')
        if shape1[0] != shape2[0] or shape1[2] != shape2[2]:
            raise ValueError('A `Match` layer should be called '
                             'on 2 inputs with same 0,2 dimensions.')

        if self.match_type in ['dot']:
            output_shape = [shape1[0], shape1[1], shape2[1], 1]
        elif self.match_type in ['mul', 'plus', 'minus']:
            output_shape = [shape1[0], shape1[1], shape2[1], shape1[2]]
        elif self.match_type in ['concat']:
            output_shape = [shape1[0], shape1[1], shape2[1], shape1[2]+shape2[2]]

        return tuple(output_shape)

    def compute_mask(self, inputs, mask=None):
        return None

    def get_config(self):
        config = {
            'normalize': self.normalize,
            'match_type': self.match_type,
        }
        base_config = super(Match, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def match(inputs, axes, normalize=False, match_type='dot', **kwargs):
    """Functional interface to the `Match` layer.
    # Arguments
        inputs: A list of input tensors (with exact 2 tensors).
        normalize: Whether to L2-normalize samples along the
            dot product axis before taking the dot product.
            If set to True, then the output of the dot product
            is the cosine proximity between the two samples.
        **kwargs: Standard layer keyword arguments.
    # Returns
        A tensor, the dot product matching matrix of the samples 
        from the inputs.
    """
    return Match(normalize=normalize, match_type=match_type, **kwargs)(inputs)

In [7]:
vocab_size = len(dico)

class Settings(object):
    text1_maxlen = 20
    text2_maxlen = 20
    embedding_dim = 300
    vocab_size = vocab_size
    
    hidden_sizes = 64
    drop_out = 0.2
    topk = 30
    
settings = Settings()

In [8]:
# 定义网络
query = Input(name='query', shape=(settings.text1_maxlen,))
doc = Input(name='doc', shape=(settings.text2_maxlen,))

embedding = Embedding(settings.vocab_size + 1,
                      settings.embedding_dim,
                      weights=[embedding_matrix],
                      input_length=settings.text1_maxlen,
                      trainable=False)

q_embed = embedding(query)
d_embed = embedding(doc)

q_rep = Bidirectional(GRU(settings.hidden_sizes, return_sequences=True, dropout=settings.drop_out))(q_embed)
d_rep = Bidirectional(GRU(settings.hidden_sizes, return_sequences=True, dropout=settings.drop_out))(d_embed)

cross = Match(match_type='dot')([q_rep, d_rep])

cross_reshape = Reshape((-1, ))(cross)

mm_k = Lambda(lambda x: K.tf.nn.top_k(x, k=settings.topk, sorted=True)[0])(cross_reshape)

pool1_flat_drop = Dropout(rate=settings.drop_out)(mm_k)

primarycaps = Reshape(target_shape=[-1, 30], name="primarycap_reshape")(pool1_flat_drop)
primarycaps = Lambda(squash, name="primarycap_squash")(primarycaps)

digitcaps = CapsuleLayer(num_capsule=2, dim_capsule=64, routings=2, name="cdigitcaps")(primarycaps)

out_ = Length(name="capsnet")(digitcaps)

model = Model(inputs=[query, doc], outputs=out_)
model.summary()   

Instructions for updating:
dim is deprecated, use axis instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
query (InputLayer)              (None, 20)           0                                            
__________________________________________________________________________________________________
doc (InputLayer)                (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 300)      25293600    query[0][0]                      
                                                                 doc[0][0]                        
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 20, 128

In [9]:
import datetime,time
from keras.optimizers import Adam

callback_list = [
    keras.callbacks.EarlyStopping(
        monitor='acc',
        patience=5
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=MODEL_PATH,
        monitor='val_acc',
        save_best_only=True
    )
]

adadelta=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['binary_accuracy'])
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

print('Train...')

model.fit([x1_train, x2_train], y_train, epochs=100, batch_size=128, callbacks=callback_list, validation_data=([x1_valid, x2_valid], y_valid))

Train...
Train on 323426 samples, validate on 40428 samples
Epoch 1/100
323426/323426 [==============================] - 536s 2ms/step - loss: 0.5681 - acc: 0.7058 - val_loss: 0.5203 - val_acc: 0.7440
Epoch 2/100
323426/323426 [==============================] - 528s 2ms/step - loss: 0.5037 - acc: 0.7517 - val_loss: 0.4777 - val_acc: 0.7647
Epoch 3/100
323426/323426 [==============================] - 528s 2ms/step - loss: 0.4681 - acc: 0.7738 - val_loss: 0.4548 - val_acc: 0.7796
Epoch 4/100
323426/323426 [==============================] - 527s 2ms/step - loss: 0.4416 - acc: 0.7894 - val_loss: 0.4393 - val_acc: 0.7900
Epoch 5/100
323426/323426 [==============================] - 526s 2ms/step - loss: 0.4208 - acc: 0.8021 - val_loss: 0.4268 - val_acc: 0.7962
Epoch 6/100
323426/323426 [==============================] - 528s 2ms/step - loss: 0.4023 - acc: 0.8116 - val_loss: 0.4351 - val_acc: 0.7949
Epoch 7/100
323426/323426 [==============================] - 529s 2ms/step - loss: 0.3870 - ac

In [10]:
y_vec_pred = model.predict([x1_test, x2_test], batch_size=128)

In [11]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

def to_one_hot(prob):
    prob_shape = prob.shape
    y_pred = np.zeros(prob_shape)
    
    i = 0
    for item in prob:
        if item[0] < item[1]:
            y_pred[i] = np.array([0, 1])
        else:
            y_pred[i] = np.array([1, 0])
        i += 1
    return y_pred

y_pred = to_one_hot(y_vec_pred)
print(y_pred)

[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [12]:
report = precision_recall_fscore_support(y_test, y_pred, average='macro')
    
def print_prf(result):
    precision = str(round(result[0] * 100, 2)) + "%"
    recall = str(round(result[1] * 100, 2)) + "%"
    f1 = str(round(result[2] * 100, 2)) + "%"
    
    strResult="Precision:"+precision+"\tRecall:"+recall+"\tF-score:"+f1
    print(strResult)
    
print_prf(report)

Precision:80.12%	Recall:81.7%	F-score:80.55%


In [13]:
model.evaluate([x1_test, x2_test], y_test)

40429/40429 [==============================] - 96s 2ms/step


[0.4961291324812493, 0.8129313116866312]